# 字典特征抽取

In [1]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import jieba
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
def dictvec():
    """
    字典数据抽取
    :return: None
    """
    # 实例化
    # sparse改为True,输出的是每个不为零位置的坐标，稀疏矩阵可以节省存储空间
    dict = DictVectorizer(sparse=False)  # 把sparse改为True看看
    # 矩阵中存在大量的0，sparse存储只记录非零位置，节省空间
    # 调用fit_transform，可以理解为自适应的一种转换
    data = dict.fit_transform([{'city': '北京', 'temperature': 100},
                               {'city': '上海', 'temperature': 60},
                               {'city': '深圳', 'temperature': 30}])
    print(dict)
    print('-' * 30)
    print(data)
    print('-' * 50)
    print(dict.get_feature_names_out())  # 字典中的一些类别数据，分别进行转换成特征
    print('-' * 50)
    print(dict.inverse_transform(data))  # 去看每个特征代表的含义，逆转回去

    return None


dictvec()

DictVectorizer(sparse=False)
------------------------------
[[  0.   1.   0. 100.]
 [  1.   0.   0.  60.]
 [  0.   0.   1.  30.]]
--------------------------------------------------
['city=上海' 'city=北京' 'city=深圳' 'temperature']
--------------------------------------------------
[{'city=北京': np.float64(1.0), 'temperature': np.float64(100.0)}, {'city=上海': np.float64(1.0), 'temperature': np.float64(60.0)}, {'city=深圳': np.float64(1.0), 'temperature': np.float64(30.0)}]


上面的结果中，``print(data)`` 可以看出，其实是 one-hot 编码。

# 文本特征抽取

In [3]:
def couvec():
    # 实例化CountVectorizer
    # max_df, min_df整数：指每个词的所有文档词频数不小于最小值，出现该词的文档数目小于等于max_df
    # max_df, min_df小数：每个词的次数／所有文档数量
    # 参考： https://dropsong.github.io/posts/c434ad22.html  Document Frequency
    # min_df=2
    vector = CountVectorizer()

    # 调用fit_transform输入并转换数据

    res = vector.fit_transform(
        ["life is  short,i like python life",  # dislike 没有出现，is 出现了 1 次，...
         "life is too long,i dislike python",
         "life is short"])

    # 打印结果,把每个词都分离了
    print(vector.get_feature_names_out())
    print('-'*30)
    print(res)
    print('-'*30)
    print(type(res))
    # 对照feature_names，标记每个词出现的次数
    print('to array:')
    print(res.toarray())
    print('inverse_transform:')
    print(vector.inverse_transform(res))


couvec()

['dislike' 'is' 'life' 'like' 'long' 'python' 'short' 'too']
------------------------------
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 14 stored elements and shape (3, 8)>
  Coords	Values
  (0, 2)	2
  (0, 1)	1
  (0, 6)	1
  (0, 3)	1
  (0, 5)	1
  (1, 2)	1
  (1, 1)	1
  (1, 5)	1
  (1, 7)	1
  (1, 4)	1
  (1, 0)	1
  (2, 2)	1
  (2, 1)	1
  (2, 6)	1
------------------------------
<class 'scipy.sparse._csr.csr_matrix'>
to array:
[[0 1 2 1 0 1 1 0]
 [1 1 1 0 1 1 0 1]
 [0 1 1 0 0 0 1 0]]
inverse_transform:
[array(['life', 'is', 'short', 'like', 'python'], dtype='<U7'), array(['life', 'is', 'python', 'too', 'long', 'dislike'], dtype='<U7'), array(['life', 'is', 'short'], dtype='<U7')]


In [4]:
def countvec():
    """
    对文本进行特征值化,单个汉字单个字母不统计，因为单个汉字字母没有意义
    :return: None
    """
    cv = CountVectorizer()

    data = cv.fit_transform(["人生苦短，我 喜欢 python python", "人生漫长，不用 python"])

    print(cv.get_feature_names_out())

    print(data)
    print(data.toarray())

    return None


countvec()


['python' '不用' '人生漫长' '人生苦短' '喜欢']
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 6 stored elements and shape (2, 5)>
  Coords	Values
  (0, 3)	1
  (0, 4)	1
  (0, 0)	2
  (1, 0)	1
  (1, 2)	1
  (1, 1)	1
[[2 0 0 1 1]
 [1 1 1 0 0]]


In [5]:
def cutword():
    con1 = jieba.cut("今天很残酷，明天更残酷，后天很美好，但绝对大部分是死在明天晚上，所以每个人不要放弃今天。")

    con2 = jieba.cut("我们看到的从很远星系来的光是在几百万年之前发出的，这样当我们看到宇宙时，我们是在看它的过去。")

    con3 = jieba.cut("如果只用一种方式了解某样事物，你就不会真正了解它。了解事物真正含义的秘密取决于如何将其与我们所了解的事物相联系。")

    # 转换成列表
    print(type(con1))
    print('-' * 50)
    content1 = list(con1)
    content2 = list(con2)
    content3 = list(con3)
    # print(content1, content2, content3)  #打印展示
    # 把列表转换成字符串
    print('-' * 50)
    c1 = ' '.join(content1)
    c2 = ' '.join(content2)
    c3 = ' '.join(content3)

    return c1, c2, c3


def hanzivec():
    """
    中文特征值化
    :return: None
    """
    c1, c2, c3 = cutword()

    print(c1, c2, c3)

    cv = CountVectorizer()

    data = cv.fit_transform([c1, c2, c3])

    print(cv.get_feature_names_out())

    print(data.toarray())

    return None


hanzivec()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


<class 'generator'>
--------------------------------------------------


Loading model cost 0.376 seconds.
Prefix dict has been built successfully.


--------------------------------------------------
今天 很 残酷 ， 明天 更 残酷 ， 后天 很 美好 ， 但 绝对 大部分 是 死 在 明天 晚上 ， 所以 每个 人 不要 放弃 今天 。 我们 看到 的 从 很 远 星系 来 的 光是在 几百万年 之前 发出 的 ， 这样 当 我们 看到 宇宙 时 ， 我们 是 在 看 它 的 过去 。 如果 只用 一种 方式 了解 某样 事物 ， 你 就 不会 真正 了解 它 。 了解 事物 真正 含义 的 秘密 取决于 如何 将 其 与 我们 所 了解 的 事物 相 联系 。
['一种' '不会' '不要' '之前' '了解' '事物' '今天' '光是在' '几百万年' '发出' '取决于' '只用' '后天' '含义'
 '大部分' '如何' '如果' '宇宙' '我们' '所以' '放弃' '方式' '明天' '星系' '晚上' '某样' '残酷' '每个'
 '看到' '真正' '秘密' '绝对' '美好' '联系' '过去' '这样']
[[0 0 1 0 0 0 2 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 2 0 1 0 2 1 0 0 0 1 1 0 0 0]
 [0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 3 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 1 1]
 [1 1 0 0 4 3 0 0 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 2 1 0 0 1 0 0]]


In [6]:
np.log10(1.5)

np.float64(0.17609125905568124)

In [7]:
# 规范{'l1'，'l2'}，默认='l2'
# 每个输出行都有单位范数，或者：
#
# 'l2'：向量元素的平方和为 1。当应用 l2 范数时，两个向量之间的余弦相似度是它们的点积。
#
# 'l1'：向量元素的绝对值之和为 1。参见preprocessing.normalize。

# smooth_idf布尔值，默认 = True
# 通过在文档频率上加一来平滑 idf 权重，就好像看到一个额外的文档包含集合中的每个术语恰好一次。防止零分裂。

def tfidfvec():
    """
    中文特征值化,计算tfidf值
    :return: None
    """
    c1, c2, c3 = cutword()

    print(c1, c2, c3)
    print(type([c1, c2, c3]))
    tf = TfidfVectorizer()

    data = tf.fit_transform([c1, c2, c3])

    print(tf.get_feature_names_out())
    print(type(data))
    print(data.toarray())

    return None


tfidfvec()

<class 'generator'>
--------------------------------------------------
--------------------------------------------------
今天 很 残酷 ， 明天 更 残酷 ， 后天 很 美好 ， 但 绝对 大部分 是 死 在 明天 晚上 ， 所以 每个 人 不要 放弃 今天 。 我们 看到 的 从 很 远 星系 来 的 光是在 几百万年 之前 发出 的 ， 这样 当 我们 看到 宇宙 时 ， 我们 是 在 看 它 的 过去 。 如果 只用 一种 方式 了解 某样 事物 ， 你 就 不会 真正 了解 它 。 了解 事物 真正 含义 的 秘密 取决于 如何 将 其 与 我们 所 了解 的 事物 相 联系 。
<class 'list'>
['一种' '不会' '不要' '之前' '了解' '事物' '今天' '光是在' '几百万年' '发出' '取决于' '只用' '后天' '含义'
 '大部分' '如何' '如果' '宇宙' '我们' '所以' '放弃' '方式' '明天' '星系' '晚上' '某样' '残酷' '每个'
 '看到' '真正' '秘密' '绝对' '美好' '联系' '过去' '这样']
<class 'scipy.sparse._csr.csr_matrix'>
[[0.         0.         0.21821789 0.         0.         0.
  0.43643578 0.         0.         0.         0.         0.
  0.21821789 0.         0.21821789 0.         0.         0.
  0.         0.21821789 0.21821789 0.         0.43643578 0.
  0.21821789 0.         0.43643578 0.21821789 0.         0.
  0.         0.21821789 0.21821789 0.         0.         0.        ]
 [0.         0.         0.  

# 特征处理

最简单的归一化：

In [8]:
def mm():
    """
    归一化处理
    :return: NOne
    """
    mm = MinMaxScaler(feature_range=(0, 1))

    # 三个样本，四个特征
    data = mm.fit_transform([[90, 2, 10, 40], [60, 4, 15, 45], [75, 3, 13, 46]])

    print(data)

    return None


mm()

[[1.         0.         0.         0.        ]
 [0.         1.         1.         0.83333333]
 [0.5        0.5        0.6        1.        ]]


上面这种归一化容易受极值的影响，于是我们提出标准化的思路。

标准化的特点：通过对原始数据进行变换把数据变换到均值为 0, 标准差为 1 范围内。

$$
X' = \frac{X-mean}{\sigma}
$$

In [9]:
def stand():
    """
    标准化缩放，不是标准正太分布，只均值为0，方差为1的分布
    :return:
    """
    std = StandardScaler()

    data = std.fit_transform([[1., -1., 3.], [2., 4., 2.], [4., 6., -1.]])

    print(data)

    print('mean = ')
    print(std.mean_)
    print('var = ')
    print(std.var_)

    print(std.n_samples_seen_)  # 样本数
    print('-' * 50)

    data1 = std.fit_transform([[-1.06904497, -1.35873244, 0.98058068],
                               [-0.26726124, 0.33968311, 0.39223227],
                               [1.33630621, 1.01904933, -1.37281295]])
    print(data1)  #这个并不是我们想看的，没意义
    # 均值
    print('mean = ')
    print(std.mean_)
    # 方差
    print('var = ')
    print(std.var_)
    # 样本数
    print(std.n_samples_seen_)
    return None


stand()

[[-1.06904497 -1.35873244  0.98058068]
 [-0.26726124  0.33968311  0.39223227]
 [ 1.33630621  1.01904933 -1.37281295]]
mean = 
[2.33333333 3.         1.33333333]
var = 
[1.55555556 8.66666667 2.88888889]
3
--------------------------------------------------
[[-1.06904497 -1.35873244  0.98058068]
 [-0.26726124  0.33968311  0.39223227]
 [ 1.33630621  1.01904933 -1.37281295]]
mean = 
[0. 0. 0.]
var = 
[1.         1.         1.00000001]
3


# 缺失值处理

缺失值处理方法：
- 删除：如果每列或者行数据缺失值达到一定的比例，建议放弃整行或者整列。
- 插补：可以通过缺失值每行或者每列的平均值、中位数来填充。

In [10]:
# 下面是填补，针对删除，可以用pd和np
def im():
    """
    缺失值处理
    :return:NOne
    """
    # NaN, nan,缺失值必须是这种形式，如果是？号(或者其他符号)，就要replace换成这种
    im = SimpleImputer(missing_values=np.nan, strategy='mean')

    data = im.fit_transform([[1, 2], [np.nan, 3], [7, 6], [3, 2]])

    print(data)

    return None


im()

[[1.         2.        ]
 [3.66666667 3.        ]
 [7.         6.        ]
 [3.         2.        ]]


# 特征选择

特征选择就是**从提取到的所有特征中选择部分特征**作为训练集特征，特征在**选择前和选择后可以改变值、也可以不改变值**，但是选择后的特征**维数肯定比选择前小**，毕竟只选择了其中的一部分特征。

做**特征选择**的原因：
- 冗余：部分特征的相关度高，容易消耗计算性能
  - 例如两个特征：一个身高 cm，一个个子高矮类别。这样就没必要用两个
- 噪声：部分特征对预测结果有负影响

主要方法：
- Filter(过滤式): VarianceThreshold , 删除低方差特征。
- Embedded(嵌入式)：正则化、决策树
- Wrapper(包裹式)
  - Wrapper 从初始特征集合中不断地选择特征子集，训练学习器，根据学习器的性能对子集进行评价，直到选出最佳的子集。从最终学习器的性能来看，Wrapper 比过滤式更好；但由于特征选择过程中需要多次训练学习器，因此 Wrapper 特征选择的计算开销通常比过滤式特征选择大得多。

In [11]:
def var():
    """
    特征选择-删除低方差的特征
    :return: None
    """
    # 默认只删除方差为0,threshold是方差阈值，删除比这个值小的那些特征
    var = VarianceThreshold(threshold=1)

    data = var.fit_transform([[0, 2, 0, 3],
                              [0, 1, 4, 3],
                              [0, 1, 1, 3]])

    print(data)
    # 获得剩余的特征的列编号
    print('The surport is %s' % var.get_support(True))
    return None


var()


[[0]
 [4]
 [1]]
The surport is [2]


# 降维

高维度数据容易出现的问题：特征之间通常是**线性相关**的。

PCA（主成分分析）:
- PCA是一种分析、简化数据集的技术。
- 压缩数据维数，尽可能降低原数据的维数（复杂度），代价是可能损失少量信息。
- 可以削减回归分析或者聚类分析中特征的数量

一个简单的例子，将这个二维的数据简化成一维：

![pic1](./1.png)

In [12]:
def pca():
    """
    主成分分析进行特征降维
    :return: None
    """
    # n_ components:小数 0~1 90% 业界选择 90~95%
    #
    # 整数   减少到的特征数量

    # pca = PCA(n_components=1)
    # pca = PCA(n_components=3)
    pca = PCA(n_components=0.9) # 保留 90% 的特征

    data = pca.fit_transform([[2, 8, 4, 5], [6, 3, 0, 8], [5, 4, 9, 1]])

    print(data)

    return None


pca()

[[-1.28620952e-15  3.82970843e+00]
 [-5.74456265e+00 -1.91485422e+00]
 [ 5.74456265e+00 -1.91485422e+00]]


另一个例子：

![pic2](./2.png)